<a href="https://colab.research.google.com/github/Tikquuss/electre_tri/blob/main/Electre_tri.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Useful functions**

**Necessary imports**

In [3]:
import itertools
import os
import pandas as pd

## Step 1: Determination of partial concordance indices

### **Calculation of $c_j(a, b_i)$ and $c_j(b_i, a)$**

For each criterion $j$, the partial match index between the alternative $a$ and the bi profile is given by :
* If the function $g_j$ is to be maximized:

$
c_j(a, b_i) = \left\{
    \begin{array}{ll}
        1 & \mbox{if } g_j(a) \geq  g_j(b_i) \\
        0 & \mbox{or.}
    \end{array}
\right. 
$ and $
c_j(b_i, a) = \left\{
    \begin{array}{ll}
        1 & \mbox{if } g_j(b_i) \geq  g_j(a) \\
        0 & \mbox{or.}
    \end{array}
\right.
$
<!-- * If the function $g_j$ is to be minimized :

$
c_j(a, b_i) = \left\{
    \begin{array}{ll}
        1 & \mbox{if } g_j(b_i) \geq  g_j(a) \\
        0 & \mbox{or.}
    \end{array}
\right. 
$ and $
c_j(b_i, a) = \left\{
    \begin{array}{ll}
        1 & \mbox{if } g_j(a) \geq  g_j(b_i) \\
        0 & \mbox{or.}
    \end{array}
\right.
$ -->

with $g_j(H)$ and $g_j(b_i)$ representing respectively the score of $H$ and $b_i$ on the criterion $j$. In our case, this score represents the qualitative evaluation attributed to $H$ and $b_i$. Therefore, the comparison $g_j(H) \geq g_j(b_i)$ simply means that the
qualitative value of $H$ is at least as good as that of $b_i$.

In [4]:
def get_c_j(H : list, b_i : list, j : int) :
    """returns cj(H,b_i) and cj(b_i,H) according to the above formula, all belonging to {0, 1}
    Parameters :
      - H (list or array) : food (array of size equal to the number of criteria, example [1, 10, 100, 2.1, 0] for 5 criteria)
      - b_i (list or array) : profile (same comment as for H, example : [2, 1, 1, 100, 1, 1] )
      - j (int) : criterion index (between 0 and the number of criteria)

  """
#     assert type_critere in ["max", "min"]
#     if type_critere == "max" :
    c_j_H_b_i = 1 if H[j] >= b_i[j] else 0
    c_j_b_i_H = 1 if b_i[j] >= H[j] else 0
#     if type_critere == "min" :
#         c_j_H_b_i = 1 if b_i[j] >= H[j] else 0
#         c_j_b_i_H = 1 if H[j] >= b_i[j] else 0
  
    return c_j_H_b_i, c_j_b_i_H

**Function for calculating partial concordance indices**.

In [5]:
def get_indices_de_concordance_partiels(criteria: int, aliments : dict, profiles : dict) :
    """Returns a dictionary containing for each alternative a and each criterion b_i
     the partial concordance indices c(a, bi) and c(bi, a), all real (float)

     Parameters :
        - food (dict) : dictionary containing the food (the key corresponds to the name of the food, and the value to the 
                            qualitative evaluation attributed to the food for each criteria)
              Example : food = {"food1" : [1, 2, 3, 1, 0.1, 10], 'food2': [2, 0, 1, 5, 1, 10]}
        - profiles (dict) : dictionary containing the profiles (the key corresponds to the name of the profile, and the value to the 
                            qualitative evaluation attributed to the profile for each criteria)
              Example : profiles = {"b6" : [100, 0, 0, 0, 100, 100], "b5" : [1550, 11, 0.8, 0.3, 10, 11], 
                                  "b4": [1650, 14, 1, 0.4, 7, 8], "b3": [1750, 17, 1.7, 0.5, 4, 5], 
                                  "b2" : [1850, 20, 4, 0.6, 3, 2.5], "b1" : [10000, 100, 100, 100, 0, 0]}"""
    criteria = criteria
    c = {}
    for j in range(criteria) : # there are j criteria
        c[j] = {} # initial each criteria as a dictionary
        for H, b_i in itertools.product(*[aliments.keys(), profils.keys()]) :
            c[j][H] = c[j].get(H, {})
            c[j][b_i] = c[j].get(b_i, {})
            c[j][H][b_i], c[j][b_i][H] = get_c_j(H = aliments[H], b_i = profils[b_i], j = j)
    return c

In [ ]:
# aliments = {"aliment_1" : [19,20,20,20],"aliment_2" : [50,50,60,60]}
# profiles = {  "b1" : [80,80,80,80], "b2" : [60,60,60,60],"b3" : [40,40,40,40],"b4" : [20,20,20,20] } 
# c = get_indices_de_concordance_partiels(aliments, profiles)
# c

## Step 2: Determination of global concordance index and credibility.

### get global concordance

The overall match index between the feed $H$ and the profile $b_i$ is given by the following formula:
$
C(a, b_i) = {\sum_{j=1}^{m} w_jc_j(a,b_i)}
$
and
$
C(b_i, a) = {\sum_{j=1}^{m} w_jc_j(b_i, a)}
$

where $k_j$ is the weight of criterion $j$ and $n$ the number of criteria

In [37]:
def get_indices_de_concordance_globaux( indices_de_concordance_partiels : dict, weights : list) :
    """Returns a dictionary containing for each food H and each criterion b_i
     the global concordance indices c(H, bi) and c(bi, H), all real (float)

     Parameters :
        - partial_matching_indices (dict) : dictionaries containing the partial matching indices computed as illustrated in step 1
        - weight (list) : lists of weights for each criteria (weight[j] = weight of criterion j)
              Example : weight = [1, 1, 1, 1, 2, 2]
    """
    C = {}
    for H, b_i in itertools.product(*[aliments.keys(), profils.keys()]) :
        C[H] = C.get(H, {})
        C[b_i] = C.get(b_i, {})
        C[H][b_i] = sum([ weights[j] * indices_de_concordance_partiels[j][H][b_i] for j in range(len(weights))])
        C[b_i][H] = sum([ weights[j] * indices_de_concordance_partiels[j][b_i][H] for j in range(len(weights))])
    return C

In [ ]:
# weights = [0.25, 0.25, 0.25, 0.25] 
# con_dic = get_indices_de_concordance_globaux( c, weights)
# con_dic

### non_discordance


In [74]:
def get_nd_j(H : list, b_i : list, veto: int):
    nd_j_H_b_i = 1
    nd_j_b_i_H = 1
    for j in range(len(H)):
        if(H[j] - b_i[j] >= veto):
            nd_j_b_i_H = 0
        if(b_i[j] - H[j] >= veto):
            nd_j_H_b_i = 0
    return nd_j_H_b_i, nd_j_b_i_H

In [ ]:
# ### testing

# nd_j_H_b_i, nd_j_b_i_H = get_nd_j([1,2,3,7],[2,2,2,2],5)

# nd_j_H_b_i

In [75]:
def non_discordance(aliments : dict, profiles : dict, veto: int):
    """Returns a dictionary containing for each alternative a and each criterion b_i
     the non_discordance_index c(a, bi) and c(bi, a), all real (float)"""
    criteria = len(profiles)
    c = {}
    for j in range(criteria) : # there are j criteria
        c[j] = {} # initial each criteria as a dictionary
        for H, b_i in itertools.product(*[aliments.keys(), profiles.keys()]) :
            c[j][H] = c[j].get(H, {})
            c[j][b_i] = c[j].get(b_i, {})
            c[j][H][b_i], c[j][b_i][H] = get_nd_j(H = aliments[H], b_i = profiles[b_i], veto = veto)
    return c
    

In [ ]:
#aliments = {"aliment_1" : [19,20,20,20],"aliment_2" : [30,50,60,60]}
#profils = {  "b1" : [80,80,80,80], "b2" : [60,60,60,60],"b3" : [40,40,40,40],"b4" : [20,20,20,20] } 

# veto = 60
# non_dis_dic = non_discordance(aliments, profiles, veto)
# non_dis_dic = non_dis_dic[len(non_dis_dic)-1]
# non_dis_dic

### Crediability Calculation

In [109]:
def get_crediability(con_dic: dict, non_dis_dic: dict):
#     con_dic_example
#     {'aliment_1': {'b4': 1.0, 'b3': 0.5, 'b2': 0.25, 'b1': 0.0},
#    'b4': {'aliment_1': 0.25, 'aliment_2': 0.5},
#    'b3': {'aliment_1': 0.75, 'aliment_2': 1.0},
#    'b2': {'aliment_1': 1.0, 'aliment_2': 1.0},
#    'b1': {'aliment_1': 1.0, 'aliment_2': 1.0},
#    'aliment_2': {'b4': 0.75, 'b3': 0.0, 'b2': 0.0, 'b1': 0.0}}
    cre = dict(con_dic)
    for key in con_dic.keys():
        for inner_key in con_dic[key]:
            cre[key][inner_key] =  con_dic[key][inner_key]  * non_dis_dic[key][inner_key]

    return cre

In [106]:
test = {'aliment_1': {'b4': 1.0, 'b3': 0.5, 'b2': 0.25, 'b1': 0.0},
        'b4': {'aliment_1': 0.25},
        'b3': {'aliment_1': 0.75},
        'b2': {'aliment_1': 1.0},
        'b1': {'aliment_1': 1.0}
       }

result = dict(test)

for key in test.keys():
    for key1 in test[key]:
        print(key)
        print(key1)
        print(" ")
        result[key][key1] =  test[key][key1]  * test[key1][key]

result

aliment_1
b4
 
aliment_1
b3
 
aliment_1
b2
 
aliment_1
b1
 
b4
aliment_1
 
b3
aliment_1
 
b2
aliment_1
 
b1
aliment_1
 


{'aliment_1': {'b4': 0.25, 'b3': 0.375, 'b2': 0.25, 'b1': 0.0},
 'b4': {'aliment_1': 0.0625},
 'b3': {'aliment_1': 0.28125},
 'b2': {'aliment_1': 0.25},
 'b1': {'aliment_1': 0.0}}

## Step 3: Determining the S outranking relationship

The outranking relationship is defined using the cutting index $\lambda$, called the majority threshold (usually higher than 50%), which represents the parameter determining the preference situation between the food $H$ and the profile $b_i$. Thus for food $H$ and a profile $b_i$ :
- $H$ outranks $b_i$ and we will note $H S b_i$ if and only if $C(H, b_i) \geq \lambda$.
- $b_i$ overranks $H$ and we note $b_i S H$ if and only if $C(b_i, H) \geq \lambda$.

In [10]:
def surclass(majority_threshold : float, H : str, b_i : str, crediability : dict):
    """outranking relation: returns H S b_i and b_i S H (which are all boolean: i.e. True or False)
     Parameters :
        - majority_threshold (float) : real between 0 and 1 (or 0% to 100%) representing the majority threshold
        - H (str) : name of the food
        - b_i (str) : name of the profile
        - global_agreement_indices (dict) : dictionaries containing the global agreement indices calculated as illustrated in step 2
    """
    H_S_b_i = True if (crediability[H][b_i] >= majority_threshold) or (crediability[H][b_i] > crediability[b_i][H]) else False
    b_i_S_H = True if (crediability[b_i][H] >= majority_threshold) or (crediability[b_i][H] > crediability[H][b_i]) else False
    return H_S_b_i, b_i_S_H

In [ ]:
# Test
H_S_b_k, _ = surclass(majority_threshold = 0.7, H= 'aliment_1', b_i='b4', crediability = crediability)
H_S_b_k

## Step 4: Assignment Procedures

Two procedures for assigning feed H are possible:

- **Pessimistic procedure**: For each food $H$, decrease the profile indices by $r$ until the first index $k$ such that $H S b_k$. The food $H$ is then assigned to the category $C_k$.

  If $H S b_k$ is never realized, then $H$ is assigned to the worst category, $C_1$ **(Olivier Sobri et al., page 5/24)** [1]
- **Optimistic procedure**: for each feed $H$, increase the profile indices from $1$ to the first index $k$ such that $b_k S H$ and $non(H S b_k)$. The food $H$ is then assigned to the category $C_{k-1}$.

  If $b_k S H$ and $non(H S b_k)$ is never realized, then $H$ is assigned to the best category, $C_r$ **(Olivier Sobri et al., page 5/24)** [1]

[1] Integration of the decision support method ELECTRE TRI in an open source geographic information system : Olivier Sobrie, Marc Pirlot, Florent Joerin

### **Pessimistic procedure

In [11]:
def PessimisticmajoritySorting(aliments : dict, profils : dict, crediability : dict, majority_threshold : float) :
    """ Classifies each food into a category according to the pessimistic assignment procedure
      Parameters : 
          - categories (list) : list of categories 
              Example : categories= ["A", "B", "C", "D", "E"]
          - aliments (dict) : dictionary containing the foods (the key corresponds to the name of the food, and the value to the 
                              qualitative evaluation attributed to the food for each criteria)
                      Example : foods = {"food1" : [1, 2, 3, 1, 0.1, 10], 'food2': [2, 0, 1, 5, 1, 10]}
          - crediability (dict) : dictionaries containing the global matching indices calculated as illustrated in step 2
          - majority_threshold (float) : real between 0 and 1 (or 0% to 100%) representing the majority threshold
    """
    result = {} # create an empty dic    
    r = len(profils) # there are #r classes in total   r = 4
    b = list(profils.keys()) # store keys of profiles in a list         b =  ["b1", "b2", "b3", "b4"]
    print(crediability)
    for H in aliments.keys() : # for each key in alternatives           H =  ["aliment_1", "aliment_2"]
        result[H] = b[r-1]
        for k in range(0, r, +1) : # loop r-1 times which is         K =  in range(3,-1,-1) = 3,2,1,0
            H_S_b_k, _ = surclass(majority_threshold, H, b[k], crediability) # surclass(0.7,"aliment_1",b[3],crediability)
            print("now start comparing " + " H is " + str(H) + " b[k] is " + str(b[k]))
            print("now comparing " + str(crediability[H][b[k]]) + " to " + str(majority_threshold) )
            if H_S_b_k :
                print(  str(k) + " is True")
                result[H] = b[k]
                break
    return result


In [ ]:
majority_threshold = 0.7
print(aliments)
print(profils)
result = PessimisticmajoritySorting(aliments= aliments, profils= profils, crediability = crediability, majority_threshold=majority_threshold)
result

## **Function useful for reading and writing in excel files

In [31]:
def get_profils(file_path : str, sheet_name : str = None, need_transform = False):
    """Takes an excel file containing profiles or feeds and returns the results in the format :
      - food = {"food_1": [1, 2, 3, 1, 0.1, 10], ..., 'food_n': [2, 0, 1, 5, 1, 10]} for example, if it is the food
      - profiles = {"b6" : [100, 0, 0, 0, 100, 100], ..., "b1" : [10000, 100, 100, 100, 0, 0]} for example, if it is about profiles

      Parameters:
        - file_path (str) : path of the excel file
        - sheet_name (str) name of the target excel sheet (if no sheet is specified, the first sheet is considered)
    """

    if sheet_name is None :
        content = pd.read_excel(file_path, index_col=0)
    else :
        content = pd.read_excel(file_path, index_col=0, sheet_name=sheet_name)
    indexs = content.index
    profils = {key : [] for key in indexs}
    for critere in content.keys() :
        C = content[critere]
        if(need_transform):
            for key in indexs :
                if(C[key]>=95):
                    temp = 5
                elif (C[key]>=80):
                    temp = 4
                elif (C[key]>=65):
                    temp = 3
                elif (C[key]>=50):
                    temp = 2
                else:
                    temp = 1
                profils[key].append(temp)
        else: 
            for key in indexs :
                temp = C[key]
                profils[key].append(temp)
    return profils


def get_criteres_poids(file_path : str, sheet_name : str = None):
    """Takes an excel file containing the criteria, their weight and their type (max or min) and returns the results in the format :
      - criteria = [["Energy", 'min'], ["Sat. fatty acid", 'min'], ["Sugar", 'min'], ["Sodium", 'min'], ["Protein", 'max'], ["Fiber", 'max']] for example
      - weight = [1, 1, 1, 1, 2, 2] for example

      Parameters :
        - file_path (str) : path of the excel file
        - sheet_name (str) name of the target excel sheet (if no sheet is specified, the first sheet is considered)
    """
    
    if sheet_name is None :
        content = pd.read_excel(file_path, index_col=0)
    else :
        content = pd.read_excel(file_path, index_col=0, sheet_name=sheet_name)
    weights = []
    for critere in content.keys() :
        C = content[critere]
        weights.append(C["weights"])
    return weights

def to_excel(classement, output_file : str, sheet_name : str = None):
    """Function writing the results provided by the assignment procedures in the excel files

      Parameters :
        - classification (dict) : dictionary containing for each food (key = name of the food) its category (value)
        - output_file (str) : path of the excel file in which the result will be stored
        - sheet_name (str) name of the target excel sheet (if no sheet is specified, the first sheet is considered)
    """
    index = classement.keys()
    columns = ["categories"]
    df = list(classement.values())
    df = [[a] for a in df]
    df = pd.DataFrame(df, index=index, columns = columns)
    if sheet_name is None :
        df.to_excel(output_file)
    else :
        df.to_excel(output_file, sheet_name = sheet_name)

# **Testing functions by simulation on dummy data**.

In [ ]:
categories = ["b1", "b2", "b3", "b4", "b5"]
weights = [0.2, 0.2, 0.2, 0.2, 0.2]
profiles = {"b1" : [80, 80, 80, 80, 80],"b2" : [60, 60, 60, 60, 60],"b3" : [40, 40, 40, 40, 40],"b4" : [20, 20, 20, 20, 20]}
aliments = {"aliment_1" : [20,40,60,20,40], 'aliment_2': [40, 60, 80, 60, 40]}
majority_threshold = 0.6
veto = 50

## **Step 1: Determination of partial concordance indices

In [ ]:
c = get_indices_de_concordance_partiels(criteria= 5 ,aliments = aliments, profiles = profiles)
c

## Step 2: Determination of global concordance indices

In [ ]:
con_dic =  get_indices_de_concordance_globaux( indices_de_concordance_partiels = c, weights = weights)
con_dic

## Step 3: Determination of Non_discordance

In [ ]:
non_dis_dic = non_discordance(aliments, profiles, veto = veto)
non_dis_dic = non_dis_dic[len(non_dis_dic)-1]
non_dis_dic

## Step 4: Crediability Calculation

In [ ]:
crediability = get_crediability(con_dic,non_dis_dic)
crediability

## Step 5 and 6: Determining the S outranking relationship & Assignment Procedures

In [ ]:
result = PessimisticmajoritySorting(aliments= aliments, profils= profils, crediability = crediability, majority_threshold=majority_threshold)
result

# **Testing data from files

**Repeatedly use these cells in order to progressively generate results**. 

That is : 
* specify the file paths
* Run the cells until you get and write the results in the output file
* Repeat for other file(s) 




> NB: Specify the paths to your files and make sure that the files exist and respect the format required by the reading functions (templates are present in this [google drive folder](https://https://drive.google.com/drive/folders/1rvEylqVGC4AGGeJzFpxWlpNaI3H_uUXi?usp=sharing)).

> If the file is badly encoded (with strange symbols in the names of foods for example), the reading will generate an error. 

> In my case I uploaded the files to google colab (See the icon on the top left, with an up arrow)



> After executions I downloaded the files ``output_optimist.xlsx`` and ``output_pesimite.xlsx``.




In [58]:
profils_file = "./index/profiles.xlsx"
weights_file = "./index/weights.xlsx"
aliments_file = "./index/index2022_data.xls"
output_file_pesimite =  "./index/output_pesimite.xlsx"
majority_threshold = 0.74
veto = 3

In [59]:
weights = get_criteres_poids(file_path = weights_file)
weights

[0.02, 0.03, 0.12, 0.07, 0.04, 0.02, 0.1, 0.14, 0.05, 0.24, 0.07, 0.1]

In [60]:
profils = get_profils(file_path = profils_file)
profils

{'b1': [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
 'b2': [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4],
 'b3': [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
 'b4': [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
 'b5': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [61]:
aliments = get_profils(file_path = aliments_file, need_transform = True)
aliments

{'Albania': [2, 1, 1, 4, 3, 3, 3, 2, 4, 4, 3, 3],
 'Algeria': [1, 1, 1, 3, 2, 1, 2, 2, 4, 2, 1, 1],
 'Angola': [1, 1, 1, 4, 4, 4, 1, 2, 2, 3, 1, 1],
 'Argentina': [1, 2, 1, 3, 2, 1, 2, 2, 1, 2, 2, 2],
 'Armenia': [2, 1, 2, 4, 3, 3, 2, 1, 3, 3, 3, 3],
 'Australia': [4, 5, 4, 2, 2, 2, 4, 2, 4, 4, 4, 4],
 'Austria': [5, 4, 4, 1, 1, 3, 4, 3, 4, 3, 4, 3],
 'Azerbaijan': [2, 1, 1, 4, 2, 5, 2, 2, 3, 3, 3, 2],
 'Bahamas': [2, 3, 2, 5, 4, 3, 2, 3, 3, 1, 2, 2],
 'Bahrain': [3, 1, 1, 5, 3, 1, 2, 2, 4, 4, 4, 4],
 'Bangladesh': [1, 1, 1, 4, 4, 2, 2, 1, 3, 2, 2, 1],
 'Barbados': [3, 4, 3, 4, 3, 3, 2, 2, 3, 2, 3, 2],
 'Belarus': [1, 1, 1, 4, 2, 5, 2, 1, 3, 3, 1, 1],
 'Belgium': [4, 4, 4, 1, 1, 2, 4, 2, 4, 3, 4, 3],
 'Belize': [1, 3, 1, 3, 2, 1, 2, 2, 4, 2, 2, 2],
 'Benin': [1, 2, 1, 3, 4, 4, 1, 2, 4, 2, 2, 2],
 'Bhutan': [3, 2, 3, 4, 3, 3, 2, 2, 3, 1, 1, 1],
 'Bolivia': [1, 1, 1, 4, 2, 1, 2, 1, 3, 2, 1, 1],
 'Bosnia and Herzegovina': [1, 1, 1, 4, 1, 5, 3, 2, 4, 3, 3, 2],
 'Botswana': [3, 3, 2, 4, 2, 

In [62]:
# convert into dataframe
df = pd.DataFrame(data=aliments)

#convert into excel
df.to_excel("aliments.xlsx", index=False)
print("Dictionary converted into excel...")

Dictionary converted into excel...


### Optional: order the profiles and categories
* make sure that we start from $b_6$ to $b_1$ for the profiles
* make sure that we start from $A$ to $E$ for the categories

In [63]:
# profils = {k : v for k, v in sorted(profils.items(), key=lambda b : b[0], reverse = True)} # se rassurer qu'on part de b6 à b1
# categories = sorted(categories, key = lambda c : c, reverse = False) # se rassurer qu'on part de A à E

In [64]:
c = get_indices_de_concordance_partiels(criteria= 12 ,aliments = aliments, profiles = profils)
c

{0: {'Albania': {'b1': 0, 'b2': 0, 'b3': 0, 'b4': 1, 'b5': 1},
  'b1': {'Albania': 1,
   'Algeria': 1,
   'Angola': 1,
   'Argentina': 1,
   'Armenia': 1,
   'Australia': 1,
   'Austria': 1,
   'Azerbaijan': 1,
   'Bahamas': 1,
   'Bahrain': 1,
   'Bangladesh': 1,
   'Barbados': 1,
   'Belarus': 1,
   'Belgium': 1,
   'Belize': 1,
   'Benin': 1,
   'Bhutan': 1,
   'Bolivia': 1,
   'Bosnia and Herzegovina': 1,
   'Botswana': 1,
   'Brazil': 1,
   'Brunei Darussalam': 1,
   'Bulgaria': 1,
   'Burkina Faso': 1,
   'Burma': 1,
   'Burundi': 1,
   'Cambodia': 1,
   'Cameroon': 1,
   'Canada': 1,
   'Cabo Verde': 1,
   'Central African Republic': 1,
   'Chad': 1,
   'Chile': 1,
   'China': 1,
   'Colombia': 1,
   'Comoros': 1,
   'Congo, Democratic Republic of the Congo': 1,
   'Congo, Republic of': 1,
   'Costa Rica': 1,
   "Côte d'Ivoire": 1,
   'Croatia': 1,
   'Cuba': 1,
   'Cyprus': 1,
   'Czech Republic': 1,
   'Denmark': 1,
   'Djibouti': 1,
   'Dominica': 1,
   'Dominican Republic': 

In [65]:
# convert into dataframe
df = pd.DataFrame(data=c)

#convert into excel
df.to_excel("concordance_partical.xlsx", index=False)
print("Dictionary converted into excel...")

Dictionary converted into excel...


In [111]:
con_dic =  get_indices_de_concordance_globaux( indices_de_concordance_partiels = c, weights = weights)
con_dic

{'Albania': {'b1': 0.0,
  'b2': 0.36,
  'b3': 0.6900000000000001,
  'b4': 0.85,
  'b5': 1.0000000000000002},
 'b1': {'Albania': 1.0000000000000002,
  'Algeria': 1.0000000000000002,
  'Angola': 1.0000000000000002,
  'Argentina': 1.0000000000000002,
  'Armenia': 1.0000000000000002,
  'Australia': 1.0000000000000002,
  'Austria': 1.0000000000000002,
  'Azerbaijan': 1.0000000000000002,
  'Bahamas': 1.0000000000000002,
  'Bahrain': 1.0000000000000002,
  'Bangladesh': 1.0000000000000002,
  'Barbados': 1.0000000000000002,
  'Belarus': 1.0000000000000002,
  'Belgium': 1.0000000000000002,
  'Belize': 1.0000000000000002,
  'Benin': 1.0000000000000002,
  'Bhutan': 1.0000000000000002,
  'Bolivia': 1.0000000000000002,
  'Bosnia and Herzegovina': 1.0000000000000002,
  'Botswana': 1.0000000000000002,
  'Brazil': 1.0000000000000002,
  'Brunei Darussalam': 1.0000000000000002,
  'Bulgaria': 1.0000000000000002,
  'Burkina Faso': 1.0000000000000002,
  'Burma': 1.0000000000000002,
  'Burundi': 1.0000000000

In [67]:
# convert into dataframe
df = pd.DataFrame(data=con_dic)

#convert into excel
df.to_excel("concordance_global.xlsx", index=False)
print("Dictionary converted into excel...")

Dictionary converted into excel...


In [112]:
non_dis_dic = non_discordance(aliments, profils, veto = veto)
non_dis_dic = non_dis_dic[len(non_dis_dic)-1]
non_dis_dic

{'Albania': {'b1': 0, 'b2': 0, 'b3': 1, 'b4': 1, 'b5': 1},
 'b1': {'Albania': 1,
  'Algeria': 1,
  'Angola': 1,
  'Argentina': 1,
  'Armenia': 1,
  'Australia': 1,
  'Austria': 1,
  'Azerbaijan': 1,
  'Bahamas': 1,
  'Bahrain': 1,
  'Bangladesh': 1,
  'Barbados': 1,
  'Belarus': 1,
  'Belgium': 1,
  'Belize': 1,
  'Benin': 1,
  'Bhutan': 1,
  'Bolivia': 1,
  'Bosnia and Herzegovina': 1,
  'Botswana': 1,
  'Brazil': 1,
  'Brunei Darussalam': 1,
  'Bulgaria': 1,
  'Burkina Faso': 1,
  'Burma': 1,
  'Burundi': 1,
  'Cambodia': 1,
  'Cameroon': 1,
  'Canada': 1,
  'Cabo Verde': 1,
  'Central African Republic': 1,
  'Chad': 1,
  'Chile': 1,
  'China': 1,
  'Colombia': 1,
  'Comoros': 1,
  'Congo, Democratic Republic of the Congo': 1,
  'Congo, Republic of': 1,
  'Costa Rica': 1,
  "Côte d'Ivoire": 1,
  'Croatia': 1,
  'Cuba': 1,
  'Cyprus': 1,
  'Czech Republic': 1,
  'Denmark': 1,
  'Djibouti': 1,
  'Dominica': 1,
  'Dominican Republic': 1,
  'Ecuador': 1,
  'Egypt': 1,
  'El Salvador': 1,

In [77]:
# convert into dataframe
df = pd.DataFrame(data=non_dis_dic)

#convert into excel
df.to_excel("non_dis_dic.xlsx", index=False)
print("Dictionary converted into excel...")

Dictionary converted into excel...


In [113]:
crediability = get_crediability(con_dic,non_dis_dic)
crediability

{'Albania': {'b1': 0.0,
  'b2': 0.0,
  'b3': 0.6900000000000001,
  'b4': 0.85,
  'b5': 1.0000000000000002},
 'b1': {'Albania': 1.0000000000000002,
  'Algeria': 1.0000000000000002,
  'Angola': 1.0000000000000002,
  'Argentina': 1.0000000000000002,
  'Armenia': 1.0000000000000002,
  'Australia': 1.0000000000000002,
  'Austria': 1.0000000000000002,
  'Azerbaijan': 1.0000000000000002,
  'Bahamas': 1.0000000000000002,
  'Bahrain': 1.0000000000000002,
  'Bangladesh': 1.0000000000000002,
  'Barbados': 1.0000000000000002,
  'Belarus': 1.0000000000000002,
  'Belgium': 1.0000000000000002,
  'Belize': 1.0000000000000002,
  'Benin': 1.0000000000000002,
  'Bhutan': 1.0000000000000002,
  'Bolivia': 1.0000000000000002,
  'Bosnia and Herzegovina': 1.0000000000000002,
  'Botswana': 1.0000000000000002,
  'Brazil': 1.0000000000000002,
  'Brunei Darussalam': 1.0000000000000002,
  'Bulgaria': 1.0000000000000002,
  'Burkina Faso': 1.0000000000000002,
  'Burma': 1.0000000000000002,
  'Burundi': 1.00000000000

In [114]:
# convert into dataframe
df = pd.DataFrame(data=crediability)

#convert into excel
df.to_excel("crediability.xlsx", index=False)
print("Dictionary converted into excel...")

Dictionary converted into excel...


### **1) pessimistic**

In [115]:
result = PessimisticmajoritySorting(aliments= aliments, profils= profils, crediability = crediability, majority_threshold=majority_threshold)
result

{'Albania': {'b1': 0.0, 'b2': 0.0, 'b3': 0.6900000000000001, 'b4': 0.85, 'b5': 1.0000000000000002}, 'b1': {'Albania': 1.0000000000000002, 'Algeria': 1.0000000000000002, 'Angola': 1.0000000000000002, 'Argentina': 1.0000000000000002, 'Armenia': 1.0000000000000002, 'Australia': 1.0000000000000002, 'Austria': 1.0000000000000002, 'Azerbaijan': 1.0000000000000002, 'Bahamas': 1.0000000000000002, 'Bahrain': 1.0000000000000002, 'Bangladesh': 1.0000000000000002, 'Barbados': 1.0000000000000002, 'Belarus': 1.0000000000000002, 'Belgium': 1.0000000000000002, 'Belize': 1.0000000000000002, 'Benin': 1.0000000000000002, 'Bhutan': 1.0000000000000002, 'Bolivia': 1.0000000000000002, 'Bosnia and Herzegovina': 1.0000000000000002, 'Botswana': 1.0000000000000002, 'Brazil': 1.0000000000000002, 'Brunei Darussalam': 1.0000000000000002, 'Bulgaria': 1.0000000000000002, 'Burkina Faso': 1.0000000000000002, 'Burma': 1.0000000000000002, 'Burundi': 1.0000000000000002, 'Cambodia': 1.0000000000000002, 'Cameroon': 1.000000

{'Albania': 'b3',
 'Algeria': 'b5',
 'Angola': 'b5',
 'Argentina': 'b4',
 'Armenia': 'b4',
 'Australia': 'b3',
 'Austria': 'b3',
 'Azerbaijan': 'b4',
 'Bahamas': 'b4',
 'Bahrain': 'b3',
 'Bangladesh': 'b5',
 'Barbados': 'b4',
 'Belarus': 'b4',
 'Belgium': 'b3',
 'Belize': 'b4',
 'Benin': 'b4',
 'Bhutan': 'b5',
 'Bolivia': 'b5',
 'Bosnia and Herzegovina': 'b4',
 'Botswana': 'b4',
 'Brazil': 'b4',
 'Brunei Darussalam': 'b3',
 'Bulgaria': 'b4',
 'Burkina Faso': 'b5',
 'Burma': 'b5',
 'Burundi': 'b5',
 'Cambodia': 'b4',
 'Cameroon': 'b5',
 'Canada': 'b3',
 'Cabo Verde': 'b4',
 'Central African Republic': 'b4',
 'Chad': 'b4',
 'Chile': 'b3',
 'China': 'b4',
 'Colombia': 'b4',
 'Comoros': 'b4',
 'Congo, Democratic Republic of the Congo': 'b4',
 'Congo, Republic of': 'b5',
 'Costa Rica': 'b4',
 "Côte d'Ivoire": 'b4',
 'Croatia': 'b4',
 'Cuba': 'b5',
 'Cyprus': 'b3',
 'Czech Republic': 'b4',
 'Denmark': 'b3',
 'Djibouti': 'b5',
 'Dominica': 'b4',
 'Dominican Republic': 'b4',
 'Ecuador': 'b5',


In [117]:
df = pd.DataFrame(data=result, index=[0])

df = (df.T)

print (df)

df.to_excel('experts_weights_classification.xlsx')

            0
Albania    b3
Algeria    b5
Angola     b5
Argentina  b4
Armenia    b4
...        ..
Vanuatu    b4
Venezuela  b5
Vietnam    b4
Zambia     b5
Zimbabwe   b5

[177 rows x 1 columns]


### **Write the result in the output file, on the sheet ``categotiries_pessimists``**

In [ ]:
to_excel(categotiries_pessimist, output_file_pesimite, "categotiries_pessimistes")